In [1]:
# imports 
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
import json
from langchain.agents import Tool
from langchain.tools import tool
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema import HumanMessage

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings


import pinecone


# Load environment variables from .env
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in .env file!")


In [22]:
llm = ChatOpenAI(temperature=0.7)

def suggest_workout(input_str):
    try:
        input = json.loads(input_str)
    except json.JSONDecodeError:
        return "Error: input should be a JSON string."

    sore_muscles = input.get("sore_muscles", [])
    time = input.get("time_available", 30)
    equipment = input.get("equipment", [])

    prompt = f"""
    You are a helpful and experienced fitness coach.

    A client gives you the following info (some fields might be missing):
    - Sore muscles: {sore_muscles if sore_muscles else 'No information provided'}
    - Time available (minutes): {time if time else 'No information provided'}
    - Available equipment: {equipment if equipment else 'Assume a public gym with basic equipment'}

    Please create a personalized workout plan that:
    - Avoids sore muscles if specified!
    - Fits within the available time if given
    - Uses the available equipment or assumes a public gym otherwise
    - Keeps the plan simple and motivational

    If some info is missing, make reasonable guesses.
    """

    response = llm([HumanMessage(content=prompt)])

    return response.content

In [23]:
def suggest_workout_tool(input_str):
    return suggest_workout(input_str)

workout_tool = Tool.from_function(
    suggest_workout_tool,
    name="suggest_workout",
    description="Generates a personalized workout plan given soreness, time, and equipment in JSON format"
)

In [24]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools=[workout_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

response = agent.run("I have 30 minutes and my arms are sore. What workout should I do?")
print(response)




> Entering new AgentExecutor chain...
I should consider a workout that doesn't put too much strain on my arms.
Action: suggest_workout
Action Input: {"time": 30, "soreness": "arms"}
Observation: Based on the information provided, here is a personalized 30-minute workout plan for you at a public gym with basic equipment:

1. Warm-up (5 minutes):
   - Start with a 5-minute light jog on the treadmill or around the gym to get your heart rate up and muscles warmed up.

2. Full-body circuit (20 minutes):
   - Bodyweight squats: 3 sets of 15 reps
   - Push-ups: 3 sets of 10 reps
   - Bent-over rows (using dumbbells): 3 sets of 12 reps
   - Plank: 3 sets of 30 seconds
   - Lunges (alternating legs): 3 sets of 12 reps per leg

3. Cardio finisher (5 minutes):
   - Finish your workout with 5 minutes of high-intensity interval training (HIIT) on the stationary bike or elliptical machine. Alternate between 30 seconds of high intensity and 30 seconds of low intensity for a total of 5 minutes.

Rem

In [34]:
llm = ChatOpenAI(temperature=0.8)

def suggest_meal(input_str):
    try:
        input = json.loads(input_str)
    except json.JSONDecodeError:
        return "Error: input should be a JSON string."

    cuisine = input.get("cuisine", [])
    dietary = input.get("dietary_restriction", [])
    time = input.get("time", 30)

    prompt = f"""
    You are a helpful and experienced dietician.

    A client gives you the following info (some fields might be missing):
    - Cuisine: {cuisine if cuisine else 'No information provided'}
    - Dietary Restrictions: {dietary if dietary else 'No information provided'}
    - Time Available: {time if time else 'Assume 30 minutes is available to cook'}

    Please create a personalized meal suggestion:
    - Taking into consideration their preferred cuisine and dietary restriction
    - Fits within the available time if given
    - Give specific steps along with the ingredients

    If some info is missing, make reasonable guesses.
    Along with the above I want a complete array of all ingredients needed as a grocery list template. 
    """

    response = llm([HumanMessage(content=prompt)])

    return response.content

In [35]:
def suggest_meal_tool(input_str):
    return suggest_meal(input_str)

meal_tool = Tool.from_function(
    suggest_meal_tool,
    name="suggest_meal",
    description="Generates a personalized meal plan given cuisine, dietary restrictions, and time available in JSON format"
)

In [37]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools=[meal_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

response = agent.run("I have 30 minutes and I want a chinese meal with lots of meat. What meal should I cook?")
print(response)




> Entering new AgentExecutor chain...
I should use the suggest_meal tool to generate a personalized meal plan for a Chinese meal with lots of meat in 30 minutes.
Action: suggest_meal
Action Input: {cuisine: "chinese", dietary_restrictions: "none", time_available: 30, main_protein: "meat"}
Observation: Error: input should be a JSON string.
Thought:I need to format the input as a JSON string.
Action: suggest_meal
Action Input: {"cuisine": "chinese", "dietary_restrictions": "none", "time_available": 30, "main_protein": "meat"}
Observation: Based on the client's preference for Chinese cuisine and the lack of dietary restrictions provided, I would suggest a quick and easy stir-fry dish that can be made in 30 minutes or less.

Meal Suggestion: Beef and Vegetable Stir-Fry

Ingredients:
- 1 lb thinly sliced beef (such as flank steak)
- 2 tablespoons soy sauce
- 1 tablespoon oyster sauce
- 1 tablespoon cornstarch
- 1 tablespoon vegetable oil
- 1 onion, sliced
- 1 bell pepper, sliced
- 1 cup b

In [38]:
# Combine both tools
all_tools = [workout_tool, meal_tool]

# One orchestrator agent for both
orchestrator = initialize_agent(
    tools=all_tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Try a natural language request:
query = input("Ask for a workout or a meal for today!")
response = orchestrator.run(query)
print(response)




> Entering new AgentExecutor chain...
I should consider using the suggest_meal tool to generate a personalized meal plan based on the cuisine and dietary restrictions.
Action: suggest_meal
Action Input: {"cuisine": "indian", "dietary_restrictions": "high protein"}
Observation: Based on the client's preference for Indian cuisine and the limited time available, I recommend making a quick and easy paneer tikka masala. Paneer is a type of Indian cottage cheese that is rich in protein and pairs well with a flavorful masala sauce. 

Ingredients:
- 1 block of paneer (cubed)
- 1 onion (diced)
- 1 bell pepper (diced)
- 1 cup of plain yogurt
- 2 tablespoons of tikka masala spice blend
- 1 tablespoon of garam masala
- 1 teaspoon of turmeric
- 1 teaspoon of cumin
- 1 tablespoon of minced garlic
- 1 tablespoon of minced ginger
- Salt and pepper to taste
- Fresh cilantro for garnish

Instructions:
1. In a bowl, mix together the yogurt, tikka masala spice blend, garam masala, turmeric, cumin, garli

In [2]:
import pdfplumber

pdf_path = "workoutplan.pdf"

all_text = ""
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if text:
            all_text += text + "\n"

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
chunks = text_splitter.split_text(all_text)
print(f"Number of chunks: {len(chunks)}")

Number of chunks: 239


In [4]:
embeddings = OpenAIEmbeddings()
chunk_embeddings = [embeddings.embed_query(chunk) for chunk in chunks]

/var/folders/x1/1vc0lb715f1bhmqdh6whmvjm0000gn/T/ipykernel_18642/1675750014.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [ ]:
from pinecone import Pinecone, ServerlessSpec
import uuid 

pine_cone = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pine_cone)

In [25]:
index_name = "lifting"

# look through indices 
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

index = pc.Index(index_name)


In [26]:
vectors = []
for chunk in chunks:
    vector_id = str(uuid.uuid4()) 
    vector_embedding = embeddings.embed_query(chunk)
    vectors.append({
        "id": vector_id,
        "values": vector_embedding,
        "metadata": {
            "text": chunk
        }
    })

In [28]:
index.upsert(vectors=vectors)
print(f"Uploaded {len(vectors)} chunks to Pinecone index '{index_name}'")

Uploaded 239 chunks to Pinecone index 'lifting'


In [29]:
from langchain.vectorstores import Pinecone as LC_Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

retriever = LC_Pinecone(
    index=index,
    embedding=embeddings,
    text_key="text"
).as_retriever(search_type="similarity", search_kwargs={"k": 5})


/var/folders/x1/1vc0lb715f1bhmqdh6whmvjm0000gn/T/ipykernel_18642/4084417449.py:6: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-pinecone package and should be used instead. To use it run `pip install -U :class:`~langchain-pinecone` and import as `from :class:`~langchain_pinecone import Pinecone``.
  retriever = LC_Pinecone(
